In [1]:
import requests
import pandas as pd

# 🔐 Replace this with your actual API key
API_KEY = "89c61046d8ed0ccf9ebfa3996ef443cc"

# 📡 API endpoint and parameters
url = "https://api.the-odds-api.com/v4/sports/boxing_boxing/odds"
params = {
    "apiKey": API_KEY,
    "regions": "us,uk,eu",       # Betting regions
    "markets": "h2h",            # "head-to-head" odds
    "oddsFormat": "decimal"      # You can also use 'american' or 'fractional'
}

# 🚀 Send the GET request
response = requests.get(url, params=params)

# 📦 Convert response to JSON
if response.status_code == 200:
    data = response.json()
    print(f"Received {len(data)} events.")
else:
    print("Error:", response.status_code, response.text)


Received 46 events.


In [2]:
# Print the first event to understand the structure
import json
print(json.dumps(data[0], indent=2))

{
  "id": "33ebcf4b4c433b51f81ed5375f9fe2e7",
  "sport_key": "boxing_boxing",
  "sport_title": "Boxing",
  "commence_time": "2025-08-16T15:00:00Z",
  "home_team": "Raymond Ford",
  "away_team": "Abraham Nova",
  "bookmakers": [
    {
      "key": "onexbet",
      "title": "1xBet",
      "last_update": "2025-08-05T20:42:50Z",
      "markets": [
        {
          "key": "h2h",
          "last_update": "2025-08-05T20:42:50Z",
          "outcomes": [
            {
              "name": "Abraham Nova",
              "price": 6.85
            },
            {
              "name": "Raymond Ford",
              "price": 1.1
            },
            {
              "name": "Draw",
              "price": 31.0
            }
          ]
        }
      ]
    },
    {
      "key": "virginbet",
      "title": "Virgin Bet",
      "last_update": "2025-08-05T20:42:51Z",
      "markets": [
        {
          "key": "h2h",
          "last_update": "2025-08-05T20:42:51Z",
          "outcomes": [
   

In [6]:
import pandas as pd

flat_data = []

for event in data:
    for bookmaker in event['bookmakers']:
        for market in bookmaker['markets']:
            for outcome in market['outcomes']:
                flat_data.append({
                    'event_id': event['id'],
                    'commence_time': event['commence_time'],
                    'fighter': outcome['name'],
                    'market': market['key'],
                    'decimal_odds': outcome['price'],
                    'bookmaker': bookmaker['title'],
                    'last_update': bookmaker['last_update'],
                    'home_team': event.get('home_team'),
                    'away_team': event.get('away_team')
                })

odds_df = pd.DataFrame(flat_data)


In [7]:
import sqlite3

# Connect to SQLite (creates file if doesn't exist)
conn = sqlite3.connect("boxing_odds_staging.db")

# Dump raw data into a table
odds_df.to_sql("raw_odds", conn, if_exists="replace", index=False)

# Close connection
conn.close()


In [30]:
conn = sqlite3.connect("boxing_odds_staging.db")
pd.read_sql("SELECT * FROM raw_odds LIMIT 100", conn)


,event_id,commence_time,fighter,market,decimal_odds,bookmaker,last_update,home_team,away_team
0,33ebcf4b4c433b51f81ed5375f9fe2e7,2025-08-16T15:00:00Z,Abraham Nova,h2h,6.85,1xBet,2025-08-05T20:42:50Z,Raymond Ford,Abraham Nova
1,33ebcf4b4c433b51f81ed5375f9fe2e7,2025-08-16T15:00:00Z,Raymond Ford,h2h,1.10,1xBet,2025-08-05T20:42:50Z,Raymond Ford,Abraham Nova
2,33ebcf4b4c433b51f81ed5375f9fe2e7,2025-08-16T15:00:00Z,Draw,h2h,31.00,1xBet,2025-08-05T20:42:50Z,Raymond Ford,Abraham Nova
3,33ebcf4b4c433b51f81ed5375f9fe2e7,2025-08-16T15:00:00Z,Abraham Nova,h2h,6.25,Virgin Bet,2025-08-05T20:42:51Z,Raymond Ford,Abraham Nova
4,33ebcf4b4c433b51f81ed5375f9fe2e7,2025-08-16T15:00:00Z,Raymond Ford,h2h,1.15,Virgin Bet,2025-08-05T20:42:51Z,Raymond Ford,Abraham Nova
...,...,...,...,...,...,...,...,...,...
95,f1cdd1ab69a4e0a0a075eefae1295763,2025-08-16T15:00:00Z,Filip Hrgović,h2h,1.16,BetUS,2025-08-05T20:42:52Z,Filip Hrgović,David Adeleye
96,f1cdd1ab69a4e0a0a075eefae1295763,2025-08-16T15:00:00Z,David Adeleye,h2h,5.65,Betsson,2025-08-05T20:42:51Z,Filip Hrgović,David Adeleye
97,f1cdd1ab69a4e0a0a075eefae1295763,2025-08-16T15:00:00Z,Filip Hrgović,h2h,1.11,Betsson,2025-08-05T20:42:51Z,Filip Hrgović,David Adeleye
98,f1cdd1ab69a4e0a0a075eefae1295763,2025-08-16T15:00:00Z,David Adeleye,h2h,6.40,Betfair,2025-08-05T20:42:52Z,Filip Hrgović,David Adeleye


In [29]:
import pandas as pd
import sqlite3

# Step 1: Connect to your SQLite DB and load raw data into DataFrame
conn = sqlite3.connect("boxing_odds_staging.db")
stg_df = pd.read_sql("SELECT * FROM raw_odds", conn)

# Step 2: Filter for key betting markets
key_markets = ['h2h', 'method_of_victory', 'draw']
filtered_df = stg_df[stg_df['market'].isin(key_markets)].copy()

# Step 3: Find best odds per fighter per event
best_odds = filtered_df.groupby(['event_id', 'fighter', 'market'])['decimal_odds'].max().reset_index()
best_odds = best_odds.rename(columns={'decimal_odds': 'Best_Odds'})

# Step 4: Merge back to get full info
enriched = pd.merge(
    best_odds,
    filtered_df,
    left_on=['event_id', 'fighter', 'market', 'Best_Odds'],
    right_on=['event_id', 'fighter', 'market', 'decimal_odds']
)

# Step 5: Keep only one row per fighter with best odds (e.g., first occurrence)
home = enriched[enriched['fighter'] == enriched['home_team']].drop_duplicates(
    subset=['event_id', 'fighter', 'market']
)

away = enriched[enriched['fighter'] == enriched['away_team']].drop_duplicates(
    subset=['event_id', 'fighter', 'market']
)

# Step 6: Merge one-to-one per fight (no cartesian product)
merged = pd.merge(
    home,
    away,
    on=['event_id', 'market'],
    suffixes=('_home', '_away')
)

# Step 7: Create readable fight name
merged['Fight'] = merged['home_team_home'] + " vs " + merged['away_team_home']

# Step 8: Final tidy output
final = merged[[
    'Fight',
    'fighter_home', 'Best_Odds_home', 'bookmaker_home',
    'fighter_away', 'Best_Odds_away', 'bookmaker_away',
    'market', 'commence_time_home'
]].rename(columns={
    'fighter_home': 'Fighter',
    'Best_Odds_home': 'Best_Odds',
    'bookmaker_home': 'Bookmaker',
    'fighter_away': 'Opponent',
    'Best_Odds_away': 'Opponent_Best_Odds',
    'bookmaker_away': 'Opponent_Bookmaker',
    'market': 'Market',
    'commence_time_home': 'Fight_Time'
})

# Replace market names for readability
final['Market'] = final['Market'].replace(market_map)

# Save to SQLite table
final.to_sql("BestOddsBookmaker", conn, if_exists="replace", index=False)

45

In [26]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("boxing_odds_staging.db")

# Now run your query
df = pd.read_sql("SELECT * FROM BestOddsBookmaker WHERE DATE(Fight_Time) = '2025-08-16'", conn)

print(df)

conn.close()


                            Fight          Fighter  Best_Odds   Bookmaker  \
0   Moses Itauma vs Dillian Whyte     Moses Itauma       1.12     Betfair   
1        Nick Ball vs Sam Goodman        Nick Ball       1.25     Betfair   
2    Raymond Ford vs Abraham Nova     Raymond Ford       1.15  Virgin Bet   
3  Hayato Tsutsumi vs Qais Ashfaq  Hayato Tsutsumi       1.03      Bovada   
4  Filip Hrgović vs David Adeleye    Filip Hrgović       1.20  Virgin Bet   

        Opponent  Opponent_Best_Odds Opponent_Bookmaker Market  \
0  Dillian Whyte                11.0          LowVig.ag    Win   
1    Sam Goodman                 5.1            FanDuel    Win   
2   Abraham Nova                 8.0          LowVig.ag    Win   
3    Qais Ashfaq                34.0         Virgin Bet    Win   
4  David Adeleye                 6.4            Betfair    Win   

             Fight_Time          inserted_at  
0  2025-08-16T19:00:00Z  2025-08-05 21:18:04  
1  2025-08-16T15:00:00Z  2025-08-05 21:18:04  

In [32]:
import pandas as pd
import sqlite3

# Connect to the database
conn = sqlite3.connect("boxing_odds_staging.db")

# Query fights on 16th August 2025
query = """
SELECT *
FROM BestOddsBookmaker
WHERE DATE(Fight_Time) = '2025-08-16'
"""
df = pd.read_sql(query, conn)

# Close the connection
conn.close()

# Format Fight_Time to remove leading zeroes in hour (optional)
df["Fight_Time"] = pd.to_datetime(df["Fight_Time"]).dt.strftime("%Y-%m-%d %H:%M")

# Create a clean and professional HTML table
html_output = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Boxing Odds - 16th August 2025</title>
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: #f4f4f4;
            padding: 40px;
        }}
        h2 {{
            text-align: center;
            margin-bottom: 30px;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            background-color: white;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
        }}
        th, td {{
            padding: 12px 15px;
            text-align: center;
            border-bottom: 1px solid #ddd;
        }}
        th {{
            background-color: #333;
            color: white;
        }}
        tr:hover {{
            background-color: #f1f1f1;
        }}
        .odds {{
            font-weight: bold;
        }}
    </style>
</head>
<body>
    <h2>Boxing Best Odds – 16th August 2025</h2>
    {df.to_html(index=False, classes='odds', border=0)}
</body>
</html>
"""

# Save to an HTML file with UTF-8 encoding
with open("best_odds_table.html", "w", encoding="utf-8") as f:
    f.write(html_output)
